# Chapter 16: Pretraining an Encoder: The BERT 2 Language Model
Fine-tuning BERT with the HuggingFace API

Programs from the book: [_Python for Natural Language Processing_](https://link.springer.com/book/9783031575488)

__Author__: Pierre Nugues

In [1]:
import os
import torch
import numpy as np

## Dataset

In [2]:
from datasets import load_dataset

In [3]:
imdb = load_dataset('imdb')

In [4]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

We can extract a small dataset for testing the whole pipeline

In [5]:
SMALL = False

In [6]:
if SMALL:
    imdb['train'] = imdb['train'].select(range(0, 25000, 100))
    imdb['test'] = imdb['test'].select(range(0, 25000, 100))

The dataset structure

In [7]:
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
train_set = imdb['train']

In [9]:
len(train_set)

25000

In [10]:
train_set[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [11]:
if not SMALL:
    print(train_set[12500])

{'text': 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.', 'label': 1}


In [12]:
test_set = imdb['test']

## Tokenizer

We create the tokenizer. These statements will download the model the first time we execute them. Again, it will be stored in a cache: `./~cache`

Alternatively, we can download it explicitely with the command:
```
git clone https://huggingface.co/distilbert-base-uncased
```


In [13]:
from transformers import AutoTokenizer

In [14]:
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"  # Downloaded automatically
# model_name = "~/.cache/distilbert-base-uncased" # Downloaded manually
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
train_set['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [16]:
tokens = tokenizer(train_set['text'][0])

In [17]:
tokens.keys()

dict_keys(['input_ids', 'attention_mask'])

In [18]:
tokens

{'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 383

In [19]:
tokenizer.convert_ids_to_tokens(tokens['input_ids'])

['[CLS]',
 'i',
 'rented',
 'i',
 'am',
 'curious',
 '-',
 'yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.',
 'i',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'u',
 '.',
 's',
 '.',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country',
 ',',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 '"',
 'controversial',
 '"',
 'i',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself',
 '.',
 '<',
 'br',
 '/',
 '>',
 '<',
 'br',
 '/',
 '>',
 'the',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life',
 '.',
 'in',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attention',
 '##s',
 'to',
 'making',
 'some',
 

In [20]:
tokenizer.model_max_length

512

In [21]:
tokenizer(train_set['text'][:3])

{'input_ids': [[101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 38

In [22]:
classics = [
    'Tell me, O Muse, of that hero',
    'Many cities did he visit',
    'Exiled from home am I ;']

In [23]:
tokenizer(classics, truncation=True, padding=True, max_length=512)

{'input_ids': [[101, 2425, 2033, 1010, 1051, 18437, 1010, 1997, 2008, 5394, 102], [101, 2116, 3655, 2106, 2002, 3942, 102, 0, 0, 0, 0], [101, 14146, 2013, 2188, 2572, 1045, 1025, 102, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}

We tokenize the whole dataset with `map`

In [24]:
imdb_tokenized = imdb.map(lambda x: tokenizer(x['text'],
                                              truncation=True,
                                              padding=True,
                                              max_length=128),
                          batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [25]:
imdb_tokenized['train'][:2]

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

## Fine-tuning

In [26]:
model_name

'distilbert-base-uncased'

In [27]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(model_name, num_labels=num_labels)
         .to('cpu'))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [29]:
from transformers import TrainingArguments

model_output_name = 'finetuned_imdb'
training_args = TrainingArguments(output_dir=model_output_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  weight_decay=0.01,
                                  evaluation_strategy='epoch',
                                  logging_strategy='no',  # we do not save ckeckpoints
                                  disable_tqdm=False)

/opt/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
def compute_accuracy(eval_output):
    labels_ids = eval_output.label_ids
    predictions = eval_output.predictions.argmax(axis=-1)
    return {'accuracy': np.mean(predictions == labels_ids)}

In [31]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_accuracy,
                  train_dataset=imdb_tokenized['train'],
                  eval_dataset=imdb_tokenized['test'],
                  tokenizer=tokenizer)

In [32]:
trainer.train()

  0%|          | 0/1564 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.2956322431564331, 'eval_accuracy': 0.87248, 'eval_runtime': 111.2524, 'eval_samples_per_second': 224.714, 'eval_steps_per_second': 7.029, 'epoch': 1.0}


  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.30203184485435486, 'eval_accuracy': 0.87888, 'eval_runtime': 100.2897, 'eval_samples_per_second': 249.278, 'eval_steps_per_second': 7.797, 'epoch': 2.0}
{'train_runtime': 885.1642, 'train_samples_per_second': 56.487, 'train_steps_per_second': 1.767, 'train_loss': 0.2919685968657589, 'epoch': 2.0}


TrainOutput(global_step=1564, training_loss=0.2919685968657589, metrics={'train_runtime': 885.1642, 'train_samples_per_second': 56.487, 'train_steps_per_second': 1.767, 'total_flos': 1655842483200000.0, 'train_loss': 0.2919685968657589, 'epoch': 2.0})

In [33]:
trainer.save_model()

In [34]:
metrics = trainer.evaluate()
metrics

  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.30203184485435486,
 'eval_accuracy': 0.87888,
 'eval_runtime': 101.8883,
 'eval_samples_per_second': 245.367,
 'eval_steps_per_second': 7.675,
 'epoch': 2.0}

## Prediction with the model

We load the fine-tuned model

In [35]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained(
    'finetuned_imdb')

In [36]:
input_ids = torch.LongTensor(imdb_tokenized['test']['input_ids'][-10:])
attention_mask = torch.LongTensor(
    imdb_tokenized['test']['attention_mask'][-10:])
labels = torch.LongTensor(imdb_tokenized['test']['label'][-10:])

In [37]:
input_ids

tensor([[ 101, 1045, 2034,  ..., 2191, 1000,  102],
        [ 101, 1999, 1996,  ..., 2024, 4439,  102],
        [ 101, 2307, 5469,  ..., 5469, 4038,  102],
        ...,
        [ 101, 1045, 2288,  ..., 2130, 9020,  102],
        [ 101, 2274, 2781,  ..., 2057, 1005,  102],
        [ 101, 1045, 3236,  ..., 2116, 2500,  102]])

In [38]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [39]:
finetuned_model(input_ids, attention_mask=attention_mask, labels=labels)

SequenceClassifierOutput(loss=tensor(0.7227, grad_fn=<NllLossBackward0>), logits=tensor([[-1.5788,  2.2016],
        [ 0.2768, -0.0327],
        [-1.9402,  2.4612],
        [-0.9183,  1.3621],
        [-2.1921,  2.7947],
        [-0.4745,  0.9853],
        [ 1.5657, -1.3760],
        [-1.8258,  2.2383],
        [ 1.4359, -1.2222],
        [-0.3564,  0.7587]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [40]:
finetuned_model(input_ids, attention_mask=attention_mask,
                labels=labels)[1].argmax(dim=-1)

tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 1])

In [41]:
torch.softmax(finetuned_model(
    input_ids, attention_mask=attention_mask, labels=labels)[1], dim=-1)

tensor([[0.0223, 0.9777],
        [0.5768, 0.4232],
        [0.0121, 0.9879],
        [0.0928, 0.9072],
        [0.0068, 0.9932],
        [0.1885, 0.8115],
        [0.9499, 0.0501],
        [0.0169, 0.9831],
        [0.9345, 0.0655],
        [0.2469, 0.7531]], grad_fn=<SoftmaxBackward0>)

## Prediction with a Pipeline

This includes the tokenizer

In [42]:
from transformers import pipeline

ftuned_model_name = 'finetuned_imdb'
classifier = pipeline('text-classification',
                      model=ftuned_model_name)

In [43]:
classifier(['I loved it!!!', 'I hate this movie!'])

[{'label': 'LABEL_1', 'score': 0.9746056199073792},
 {'label': 'LABEL_0', 'score': 0.9612120389938354}]

## Freezing Layers

In [44]:
model_name

'distilbert-base-uncased'

In [45]:
cnt = 0
for p in model.pre_classifier.parameters():
    print(p)
    cnt += 1
    if cnt > 10:
        break

Parameter containing:
tensor([[-0.0363, -0.0209,  0.0147,  ...,  0.0112,  0.0284, -0.0166],
        [ 0.0371, -0.0301, -0.0156,  ...,  0.0098, -0.0095, -0.0084],
        [ 0.0200,  0.0059, -0.0213,  ..., -0.0278,  0.0118,  0.0060],
        ...,
        [-0.0011, -0.0039,  0.0046,  ...,  0.0158, -0.0149, -0.0168],
        [ 0.0086,  0.0258, -0.0473,  ...,  0.0003,  0.0166,  0.0058],
        [-0.0187, -0.0348,  0.0118,  ..., -0.0348,  0.0092,  0.0339]],
       device='mps:0', requires_grad=True)
Parameter containing:
tensor([-9.5867e-04,  5.3423e-04,  7.5653e-04,  3.8816e-04,  3.2903e-04,
        -1.2096e-03, -1.0410e-03,  5.9763e-04,  5.5924e-04,  1.1421e-03,
         1.9520e-04,  5.4229e-04,  2.3733e-04, -4.2607e-05,  5.9463e-06,
        -3.5530e-05,  1.2875e-03,  4.4700e-05,  6.0970e-04,  5.1731e-04,
        -9.0756e-05, -6.8657e-05,  1.8123e-04,  5.6887e-04,  5.8761e-04,
         5.3568e-04,  1.0709e-05,  6.3489e-04,  1.3311e-04,  9.2429e-04,
        -1.4909e-05,  4.3062e-04,  3.9120

In [46]:
def count_parameters(model):
    total = 0
    trainable = 0
    for params in model.parameters():
        total += params.numel()
        if params.requires_grad:
            trainable += params.numel()
    return {'total': total, 'trainable': trainable}

In [47]:
count_parameters(model)

{'total': 66955010, 'trainable': 66955010}

We reload the original model and we freeze some layers

In [48]:
from transformers import AutoModelForSequenceClassification

num_labels = 2
model_freezed = (AutoModelForSequenceClassification
                 .from_pretrained(model_name, num_labels=num_labels)
                 .to('cpu'))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
model_freezed

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [50]:
count_parameters(model_freezed)

{'total': 66955010, 'trainable': 66955010}

In [51]:
count_parameters(model_freezed.pre_classifier)

{'total': 590592, 'trainable': 590592}

In [52]:
count_parameters(model_freezed.classifier)

{'total': 1538, 'trainable': 1538}

In [53]:
for param in model_freezed.parameters():
    param.requires_grad = False

In [54]:
for param in model_freezed.pre_classifier.parameters():
    param.requires_grad = True

for param in model_freezed.classifier.parameters():
    param.requires_grad = True

In [55]:
count_parameters(model_freezed)

{'total': 66955010, 'trainable': 592130}

This is to be compared with the original model

In [56]:
count_parameters(model)

{'total': 66955010, 'trainable': 66955010}

In [57]:
from transformers import Trainer

trainer = Trainer(model=model_freezed, args=training_args,
                  compute_metrics=compute_accuracy,
                  train_dataset=imdb_tokenized['train'],
                  eval_dataset=imdb_tokenized['test'],
                  tokenizer=tokenizer)

In [58]:
trainer.train()

  0%|          | 0/1564 [00:00<?, ?it/s]

  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.5368176698684692, 'eval_accuracy': 0.77008, 'eval_runtime': 93.2873, 'eval_samples_per_second': 267.989, 'eval_steps_per_second': 8.383, 'epoch': 1.0}


  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.5060358047485352, 'eval_accuracy': 0.77668, 'eval_runtime': 92.5383, 'eval_samples_per_second': 270.158, 'eval_steps_per_second': 8.451, 'epoch': 2.0}
{'train_runtime': 402.2053, 'train_samples_per_second': 124.315, 'train_steps_per_second': 3.889, 'train_loss': 0.5660996302924193, 'epoch': 2.0}


TrainOutput(global_step=1564, training_loss=0.5660996302924193, metrics={'train_runtime': 402.2053, 'train_samples_per_second': 124.315, 'train_steps_per_second': 3.889, 'total_flos': 1655842483200000.0, 'train_loss': 0.5660996302924193, 'epoch': 2.0})

In [59]:
metrics = trainer.evaluate()
metrics

  0%|          | 0/782 [00:00<?, ?it/s]

{'eval_loss': 0.5060358047485352,
 'eval_accuracy': 0.77668,
 'eval_runtime': 92.7522,
 'eval_samples_per_second': 269.535,
 'eval_steps_per_second': 8.431,
 'epoch': 2.0}